### Inicialização da API do Cartola FC e Configuração do Pandas

In [1]:
import cartolafc
import pandas as pd
from difflib import get_close_matches
import json
import time
import requests
from functools import lru_cache

pd.set_option('display.max_columns', 50)            # permite a visualização de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os números aparecerem com duas casas decimais

# Cria uma instância da API
api = cartolafc.Api(attempts=5)

2026-01-29 23:49:09,111 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [2]:
# Lista fixa de IDs dos participantes (lista de int)
ids_participantes = [
    # 212042, 13951133, 1747619, 25811332
    # 20696550, 30267301, 51010813, 5823700
    # 29228373, 7017989, 3851966, 13913874,
    186283, 19033717, 117598, 24856400,
    # 18642587, 18344271, 14124559, 36359,
    # 18223508, 44810918, 387186, 18346776,
    # 49355335, 24468241, 28741323, 47544767,
    # 1273719, 479510, 13707047, 528730    
]

In [3]:
HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json,text/plain,*/*",
    "Referer": "https://cartola.globo.com/",
}

@lru_cache(maxsize=5000)
def nome_time_por_id_api(time_id: int, timeout=15) -> str:
    endpoints = [
        f"https://api.cartolafc.globo.com/time/id/{time_id}",
        f"https://api.cartolafc.globo.com/time/{time_id}",
    ]

    for url in endpoints:
        for tentativa in range(3):
            try:
                r = requests.get(url, headers=HEADERS, timeout=timeout)
                if r.status_code == 429:
                    time.sleep(0.8 + tentativa * 0.8)
                    continue
                if r.status_code != 200:
                    break

                data = r.json()
                if isinstance(data, dict):
                    if isinstance(data.get("time"), dict) and isinstance(data["time"].get("nome"), str):
                        return data["time"]["nome"]
                    if isinstance(data.get("nome"), str):
                        return data["nome"]
                break
            except Exception:
                time.sleep(0.5)
                continue

    return f"Time {time_id}"


In [4]:
# Base com todos os participantes
if not isinstance(ids_participantes, list) or not ids_participantes:
    raise ValueError("ids_participantes precisa ser uma lista de IDs")

ids_participantes = list(dict.fromkeys(ids_participantes))

df_base = pd.DataFrame({"time_id": ids_participantes}).drop_duplicates()
df_base["Time"] = df_base["time_id"].apply(nome_time_por_id_api)

df_base = df_base.set_index("time_id").sort_index()

# Dicionario Nome -> ID (compatibilidade com codigo legado)
ids_times_dict = {row["Time"]: row["time_id"] for _, row in df_base.reset_index().iterrows()}

# Links para o Excel
df_urls = pd.DataFrame({
    "Nome do Time": df_base["Time"].values,
    "ID do Time": df_base.index.values,
})

df_urls["Link do Time"] = df_urls["ID do Time"].apply(
    lambda x: f"https://cartola.globo.com/#!/time/{x}"
)

df_urls = df_urls[["Nome do Time", "ID do Time", "Link do Time"]]

caminho_excel = "links_times_cartola_libertadores.xlsx"
df_urls.to_excel(caminho_excel, index=False)
print(f"? Arquivo salvo com sucesso: {caminho_excel}")

display(df_base)
display(df_urls)

? Arquivo salvo com sucesso: links_times_cartola_libertadores.xlsx


,Time
time_id,
117598,A Lenda Super Vasco F.c
186283,FBC Colorado
19033717,Mau Humor F.C.
24856400,Grêmio imortal 36


,Nome do Time,ID do Time,Link do Time
0,A Lenda Super Vasco F.c,117598,https://cartola.globo.com/#!/time/117598
1,FBC Colorado,186283,https://cartola.globo.com/#!/time/186283
2,Mau Humor F.C.,19033717,https://cartola.globo.com/#!/time/19033717
3,Grêmio imortal 36,24856400,https://cartola.globo.com/#!/time/24856400


In [5]:
nomes_times = [
                'Tatols Beants F.C',
                'JV5 Tricolor Gaúcho',
                'JUV. KP',
                'SERGRILLO',
                
                'Dom Camillo68',
                'Máquina Laranjja',
                'LISI GREMISTA',
                'S.E.R. GRILLO',

                'cartola scheuer17',
                'dasdoresfc',
                'Bandoleros FCS',
                'seralex',

                'A Lenda Super Vasco F.c',
                'FBC Colorado',
                'Grêmio imortal 36', 
                'Mau Humor F.C.', 

                'KillerColorado',  
                'Paulo Virgili FC',
                'FÚRIA LEON',
                'Fedato Futebol Clube',

                'DM Studio',
                'Rolo Compressor ZN',
                'AZURRA82',
                'lsauer fc',

                'Grêmio imortal 37',  
                'Tabajara de Inhaua PB1',
                'TORRESMO COM PINGA PRO26.1',
                'A Lenda Super Vascão f.c',

                'TEAM LOPES 99', 
                'Gremiomaniasm',  
                'Texas Club 2026',
                'Super Vasco f.c' 
                ]


# Dicionário para armazenar os IDs dos times
ids_times = {}

# Função para buscar o ID de um time pelo nome usando correspondência flexível
def buscar_id_time(nome_time):
    try:
        times = api.times(query=nome_time)
        nomes_api = [time.nome for time in times]
        nome_proximo = get_close_matches(nome_time, nomes_api, n=1, cutoff=0.6)
        if nome_proximo:
            for time in times:
                if time.nome == nome_proximo[0]:
                    return time.id
    except cartolafc.errors.CartolaFCError as e:
        print(f"Erro ao buscar ID para o time {nome_time}: {e}")
    return None

# Buscar os IDs dos times pelo nome
for nome in nomes_times:
    time_id = buscar_id_time(nome)
    if time_id:
        ids_times[nome] = time_id
    else:
        print(f"Não foi possível encontrar o ID para o time {nome}")

print(nomes_times)

['Tatols Beants F.C', 'JV5 Tricolor Gaúcho', 'JUV. KP', 'SERGRILLO', 'Dom Camillo68', 'Máquina Laranjja', 'LISI GREMISTA', 'S.E.R. GRILLO', 'cartola scheuer17', 'dasdoresfc', 'Bandoleros FCS', 'seralex', 'A Lenda Super Vasco F.c', 'FBC Colorado', 'Grêmio imortal 36', 'Mau Humor F.C.', 'KillerColorado', 'Paulo Virgili FC', 'FÚRIA LEON', 'Fedato Futebol Clube', 'DM Studio', 'Rolo Compressor ZN', 'AZURRA82', 'lsauer fc', 'Grêmio imortal 37', 'Tabajara de Inhaua PB1', 'TORRESMO COM PINGA PRO26.1', 'A Lenda Super Vascão f.c', 'TEAM LOPES 99', 'Gremiomaniasm', 'Texas Club 2026', 'Super Vasco f.c']


In [6]:
# Gerar DataFrame com nome do time, ID e URL
df_urls = pd.DataFrame([
    {"Nome do Time": nome, "ID do Time": time_id, "Link do Time": f"https://cartola.globo.com/#!/time/{time_id}"}
    for nome, time_id in ids_times.items()
])

# Caminho e nome do arquivo Excel
caminho_excel = "links_times_cartola_libertadores.xlsx"

# Salvar o DataFrame em Excel
df_urls.to_excel(caminho_excel, index=False)

print(f"✅ Arquivo salvo com sucesso: {caminho_excel}")


# Exibir como tabela
display(df_urls)

✅ Arquivo salvo com sucesso: links_times_cartola_libertadores.xlsx


,Nome do Time,ID do Time,Link do Time
0,Tatols Beants F.C,212042,https://cartola.globo.com/#!/time/212042
1,JV5 Tricolor Gaúcho,1747619,https://cartola.globo.com/#!/time/1747619
2,JUV. KP,13951133,https://cartola.globo.com/#!/time/13951133
3,SERGRILLO,25811332,https://cartola.globo.com/#!/time/25811332
4,Dom Camillo68,20696550,https://cartola.globo.com/#!/time/20696550
5,Máquina Laranjja,30267301,https://cartola.globo.com/#!/time/30267301
6,LISI GREMISTA,51010813,https://cartola.globo.com/#!/time/51010813
7,S.E.R. GRILLO,5823700,https://cartola.globo.com/#!/time/5823700
8,cartola scheuer17,3851966,https://cartola.globo.com/#!/time/3851966
9,dasdoresfc,7017989,https://cartola.globo.com/#!/time/7017989


In [7]:
# ===============================
#  FASE (dinamica) + PARCIAIS DA RODADA EM ANDAMENTO
# ===============================
import time, requests, pandas as pd

FASE_INICIO = 1          # primeira rodada da fase
FASE_LIMITE = 6          # ultima rodada possivel da fase
CAP_MULT    = 1.5
PER_REQ_SLEEP = 1.0
MAX_RETRIES   = 3

# Sessao HTTP
sess = requests.Session()
sess.headers.update({
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json, text/plain, */*",
})

def http_status_e_rodada():
    r = sess.get("https://api.cartola.globo.com/mercado/status", timeout=20)
    r.raise_for_status()
    d = r.json()
    return int(d.get("status_mercado", 0)), int(d.get("rodada_atual", 0))

def get_parciais():
    r = sess.get("https://api.cartola.globo.com/atletas/pontuados", timeout=30)
    r.raise_for_status()
    return r.json().get("atletas", {})

def fetch_time_payload(time_id: int, rodada: int, max_retries=MAX_RETRIES):
    urls = [
        f"https://api.cartola.globo.com/time/id/{time_id}/{rodada}",
        f"https://api.cartola.globo.com/time/id/{time_id}",
    ]
    for url in urls:
        wait = 0.7
        for _ in range(max_retries):
            try:
                r = sess.get(url, timeout=30)
                st = r.status_code
                if st == 200:
                    d = r.json()
                    if not isinstance(d, dict):
                        d = {}
                    return d
                if st in (403, 429, 503):
                    time.sleep(wait); wait *= 1.8; continue
                break
            except Exception:
                time.sleep(wait); wait *= 1.8
    return {}


def fetch_partidas_rodada(rodada: int):
    try:
        r = sess.get(f"https://api.cartola.globo.com/partidas/{rodada}", timeout=30)
        if r.status_code != 200:
            return []
        data = r.json()
        if isinstance(data, dict):
            partidas = data.get("partidas", [])
            return partidas if isinstance(partidas, list) else []
    except Exception:
        pass
    return []


def clubes_que_ja_jogaram(rodada: int):
    partidas = fetch_partidas_rodada(rodada)
    clubes = set()
    for p in partidas:
        if not isinstance(p, dict):
            continue
        status = (p.get("status_transmissao_tr") or p.get("status_transmissao") or "").upper()
        encerrada = status in ("ENCERRADA", "FINALIZADA")
        if p.get("placar_oficial_mandante") is not None or p.get("placar_oficial_visitante") is not None:
            encerrada = True
        if encerrada:
            mid = p.get("clube_casa_id")
            vid = p.get("clube_visitante_id")
            try:
                if mid is not None:
                    clubes.add(int(mid))
            except Exception:
                pass
            try:
                if vid is not None:
                    clubes.add(int(vid))
            except Exception:
                pass
    return clubes


def setor_por_posicao(posicao_id: int) -> str:
    mapa = {
        1: "Goleiro",
        2: "Laterais",
        3: "Zagueiros",
        4: "Meias",
        5: "Atacantes",
        6: "Tecnico",
    }
    return mapa.get(int(posicao_id), "")


def _id_int(val):
    try:
        return int(val)
    except Exception:
        return None


def calcular_parcial_time(time_id: int, rodada: int, mapa_pontuados: dict, clubes_jogaram: set):
    data = fetch_time_payload(time_id, rodada)
    atletas = data.get("atletas") if isinstance(data, dict) else None
    if not isinstance(atletas, list):
        return 0.0

    titulares = []
    tecnico = None
    for a in atletas:
        if not isinstance(a, dict):
            continue
        pos = a.get("posicao_id")
        if pos == 6:
            tecnico = a
        else:
            titulares.append(a)

    luxo_id = data.get("reserva_luxo_id") if isinstance(data, dict) else None
    if luxo_id is not None:
        luxo_id = _id_int(luxo_id)

    # Capitao (1.5x)
    capitao_id = None
    try:
        capitao_id = data.get("capitao_id") if isinstance(data, dict) else None
        if capitao_id is None and isinstance(data.get("time"), dict):
            capitao_id = data["time"].get("capitao_id")
        capitao_id = _id_int(capitao_id)
    except Exception:
        capitao_id = None

    titulares_por_pos = {}
    for a in titulares:
        pos = a.get("posicao_id")
        titulares_por_pos.setdefault(pos, []).append(a)

    atletas_em_jogo = list(titulares)

    reservas = data.get("reservas", []) if isinstance(data, dict) else []
    if isinstance(reservas, list):
        for r in reservas:
            if not isinstance(r, dict):
                continue
            # reserva de luxo nao entra como banco
            if luxo_id is not None and _id_int(r.get("atleta_id")) == luxo_id:
                continue
            # banco so entra se reserva tiver pontuacao parcial positiva
            rid = _id_int(r.get("atleta_id"))
            if rid is None or rid not in mapa_pontuados:
                continue
            if float(mapa_pontuados.get(rid, 0.0)) <= 0.0:
                continue
            pos = r.get("posicao_id")
            if pos is None:
                continue
            candidatos = titulares_por_pos.get(pos, [])
            titular_sub = None
            for t in candidatos:
                tid = _id_int(t.get("atleta_id"))
                if tid is None:
                    continue
                clube_tid = _id_int(t.get("clube_id"))
                if tid not in mapa_pontuados and (clube_tid in clubes_jogaram):
                    titular_sub = t
                    break
            if titular_sub is None:
                continue
            try:
                atletas_em_jogo.remove(titular_sub)
            except ValueError:
                pass
            atletas_em_jogo.append(r)

    # Reserva de luxo
    luxo_capitao = False
    if luxo_id is not None:
        ids_em_jogo = set()
        for a in atletas_em_jogo:
            ids_em_jogo.add(_id_int(a.get("atleta_id")))
        if luxo_id not in ids_em_jogo:
            luxo_obj = None
            if isinstance(reservas, list):
                for r in reservas:
                    if isinstance(r, dict) and _id_int(r.get("atleta_id")) == luxo_id:
                        luxo_obj = r
                        break
            if isinstance(luxo_obj, dict):
                p_luxo = float(mapa_pontuados.get(luxo_id, 0.0))
                if p_luxo > 0:
                    setor_luxo = setor_por_posicao(luxo_obj.get("posicao_id"))
                    candidatos_setor = []
                    for a in atletas_em_jogo:
                        if setor_por_posicao(a.get("posicao_id")) == setor_luxo:
                            candidatos_setor.append(a)
                    if candidatos_setor:
                        # luxo so entra se TODOS do setor ja jogaram
                        todos_setor_jogaram = True
                        for c in candidatos_setor:
                            clube_c = _id_int(c.get("clube_id"))
                            if clube_c not in clubes_jogaram:
                                todos_setor_jogaram = False
                                break
                        if todos_setor_jogaram:
                            def pts(a):
                                try:
                                    return float(mapa_pontuados.get(int(a.get("atleta_id")), 0.0))
                                except Exception:
                                    return 0.0
                            pior_pts = None
                            piores = []
                            for c in candidatos_setor:
                                v = pts(c)
                                if pior_pts is None or v < pior_pts:
                                    pior_pts = v
                                    piores = [c]
                                elif v == pior_pts:
                                    piores.append(c)
                            pior = None
                            if len(piores) > 1 and capitao_id is not None:
                                for c in piores:
                                    if _id_int(c.get("atleta_id")) == capitao_id:
                                        pior = c
                                        break
                            if pior is None:
                                pior = piores[0] if piores else None
                            if pior is not None and p_luxo > pts(pior):
                                if capitao_id is not None and _id_int(pior.get("atleta_id")) == capitao_id:
                                    luxo_capitao = True
                                try:
                                    atletas_em_jogo.remove(pior)
                                except ValueError:
                                    pass
                                atletas_em_jogo.append(luxo_obj)

    total = 0.0
    for a in atletas_em_jogo:
        try:
            aid = int(a.get("atleta_id"))
        except Exception:
            continue
        total += float(mapa_pontuados.get(aid, 0.0))

    # Bonus do capitao (50%)
    if capitao_id is not None:
        cap_pts = float(mapa_pontuados.get(capitao_id, 0.0))
        if cap_pts != 0.0:
            ids_em_jogo = set()
            for a in atletas_em_jogo:
                ids_em_jogo.add(_id_int(a.get("atleta_id")))
            if capitao_id in ids_em_jogo:
                total += cap_pts * 0.5
            elif luxo_capitao and luxo_id is not None and luxo_id in ids_em_jogo:
                luxo_pts = float(mapa_pontuados.get(luxo_id, 0.0))
                if luxo_pts != 0.0:
                    total += luxo_pts * 0.5

    if isinstance(tecnico, dict):
        try:
            tid = int(tecnico.get("atleta_id"))
            total += float(mapa_pontuados.get(tid, 0.0))
        except Exception:
            pass

    return round(total, 2)

# ---- 0) Determinar FASE_FIM dinamicamente ----
status_http, rodada_http = http_status_e_rodada()
try:
    rodada_api = api.mercado().rodada_atual
except Exception:
    rodada_api = rodada_http

rod_ref = rodada_http or rodada_api

if status_http == 2:
    FASE_FIM = max(FASE_INICIO, min(FASE_LIMITE, rod_ref))
    RODADAS_CONCLUIDAS_FIM = FASE_FIM - 1
elif status_http == 1:
    FASE_FIM = max(FASE_INICIO, min(FASE_LIMITE, rod_ref - 1))
    RODADAS_CONCLUIDAS_FIM = FASE_FIM
else:
    FASE_FIM = max(FASE_INICIO, min(FASE_LIMITE, rod_ref))
    RODADAS_CONCLUIDAS_FIM = FASE_FIM

print(f"Status={status_http} | rodada_http={rodada_http} | FASE_INICIO={FASE_INICIO} | FASE_FIM(dinamico)={FASE_FIM} | fechadas ate {RODADAS_CONCLUIDAS_FIM}")

# 1) Monta DF com rodadas CONCLUIDAS da fase
dados = {}
for nome, time_id in ids_times.items():
    pontuacoes = {}
    if RODADAS_CONCLUIDAS_FIM >= FASE_INICIO:
        for r in range(FASE_INICIO, RODADAS_CONCLUIDAS_FIM + 1):
            try:
                t = api.time(time_id=int(time_id), rodada=r)
                pontuacoes[r] = t.ultima_pontuacao
            except Exception:
                pontuacoes[r] = None
    dados[nome] = pd.Series({f"Rodada {k}": v for k, v in pontuacoes.items()})

df_pontuacoes = pd.DataFrame.from_dict(dados, orient="index")

todas = [f"Rodada {i}" for i in range(FASE_INICIO, FASE_FIM + 1)]
df_pontuacoes = df_pontuacoes.reindex(columns=todas)
df_pontuacoes = df_pontuacoes.apply(pd.to_numeric, errors="coerce")

# 2) Injeta PARCIAIS se a rodada atual estiver em andamento E dentro da fase
col_atual = f"Rodada {rod_ref}"
if status_http == 2 and (FASE_INICIO <= rod_ref <= FASE_FIM):
    print(f"Rodada {rod_ref} em andamento: aplicando parciais")
    parciais_raw = get_parciais()
    mapa_pontuados = {}
    if isinstance(parciais_raw, dict):
        for k, v in parciais_raw.items():
            try:
                mapa_pontuados[int(k)] = float(v.get("pontuacao") or 0.0)
            except Exception:
                continue

    if mapa_pontuados:
        clubes_jogaram = clubes_que_ja_jogaram(rod_ref)
        for nome_time, time_id in ids_times.items():
            try:
                time_id = int(time_id)
                total = calcular_parcial_time(time_id, rod_ref, mapa_pontuados, clubes_jogaram)
                if col_atual not in df_pontuacoes.columns:
                    df_pontuacoes[col_atual] = pd.NA
                df_pontuacoes.loc[nome_time, col_atual] = round(total, 2)
                time.sleep(PER_REQ_SLEEP)
            except Exception:
                time.sleep(PER_REQ_SLEEP)
    else:
        print("Parciais vazias agora; tente novamente em instantes.")
else:
    print("Sem parciais para aplicar nesta fase (status != 2 ou fora do intervalo).")

# 3) Recria marcador de lider (opcional)
try:
    lider = df_pontuacoes.apply(pd.to_numeric, errors="coerce").idxmax(axis=0)
    df_pontuacoes.loc["Lider_Rodada"] = lider
except Exception as e:
    print("Nao foi possivel recalcular 'Lider_Rodada':", e)

# 4) Para EXPORTAR (se quiser zeros), use uma COPIA:
df_export = df_pontuacoes.fillna(0.0)

# Visualizacao
try:
    display(df_pontuacoes.T)
except Exception:
    print(df_pontuacoes.T.head())


Status=2 | rodada_http=1 | FASE_INICIO=1 | FASE_FIM(dinamico)=1 | fechadas ate 0
Rodada 1 em andamento: aplicando parciais


,Tatols Beants F.C,JV5 Tricolor Gaúcho,JUV. KP,SERGRILLO,Dom Camillo68,Máquina Laranjja,LISI GREMISTA,S.E.R. GRILLO,cartola scheuer17,dasdoresfc,Bandoleros FCS,seralex,A Lenda Super Vasco F.c,FBC Colorado,Grêmio imortal 36,Mau Humor F.C.,KillerColorado,Paulo Virgili FC,FÚRIA LEON,Fedato Futebol Clube,DM Studio,Rolo Compressor ZN,AZURRA82,lsauer fc,Grêmio imortal 37,Tabajara de Inhaua PB1,TORRESMO COM PINGA PRO26.1,A Lenda Super Vascão f.c,TEAM LOPES 99,Gremiomaniasm,Texas Club 2026,Super Vasco f.c,Lider_Rodada
Rodada 1,66.86,47.86,54.60,49.00,72.70,49.76,47.86,84.86,61.56,72.86,74.06,60.16,68.06,53.66,64.70,73.96,72.45,84.26,54.16,63.90,60.20,59.25,38.26,62.56,57.60,61.96,73.76,54.10,58.96,57.45,68.06,42.96,S.E.R. GRILLO


### Organização dos Grupos da Fase 1 da Libertadores e Exportação para Excel

In [8]:
# Dicionário com ID -> Nome do time (gerado anteriormente)
nomes_por_id = {
    212042: 'Tatols Beants F.C',
    1747619: 'JV5 Tricolor Gaúcho',
    13951133: 'JUV. KP',
    25811332: 'SERGRILLO',

    20696550: 'Dom Camillo68',
    30267301: 'Máquina Laranjja',
    51010813: 'LISI GREMISTA',
    5823700: 'S.E.R. GRILLO',

    3851966: 'cartola scheuer17',
    7017989: 'dasdoresfc',
    13913874: 'Bandoleros FCS',
    29228373: 'seralex',

    117598: 'A Lenda Super Vasco F.c',
    186283: 'FBC Colorado',
    24856400: 'Grêmio imortal 36',
    19033717: 'Mau Humor F.C.',    

    36359: 'KillerColorado',
    14124559: 'Paulo Virgili FC',
    18344271: 'FÚRIA LEON',
    18642587: 'Fedato Futebol Clube',

    387186: 'DM Studio',
    18223508: 'Rolo Compressor ZN',
    18346776: 'AZURRA82',
    44810918: 'lsauer fc',

    24468241: 'Grêmio imortal 37',
    28741323: 'Tabajara de Inhaua PB1',
    47544767: 'TORRESMO COM PINGA PRO26.1',
    49355335: 'A Lenda Super Vascão f.c',    

    479510: 'TEAM LOPES 99',
    528730: 'Gremiomaniasm',
    1273719: 'Texas Club 2026',
    13707047: 'Super Vasco f.c'
}

# Lista com todos os dados que você passou
dados_torneio = [
    # Grupo A
    # Grupo B
    # Grupo C
    # Grupo D
    # Grupo E
    # Grupo F
    # Grupo G
    # Grupo H
    ("Grupo A", 212042),
    ("Grupo A", 1747619),
    ("Grupo A", 13951133),
    ("Grupo A", 25811332),
    ("Grupo B", 20696550),
    ("Grupo B", 30267301),
    ("Grupo B", 51010813),
    ("Grupo B", 5823700),
    ("Grupo C", 3851966),
    ("Grupo C", 7017989),
    ("Grupo C", 13913874),
    ("Grupo C", 29228373),
    ("Grupo D", 117598),
    ("Grupo D", 186283),
    ("Grupo D", 24856400),
    ("Grupo D", 19033717),
    ("Grupo E", 36359),
    ("Grupo E", 14124559),
    ("Grupo E", 18344271),
    ("Grupo E", 18642587),
    ("Grupo F", 387186),
    ("Grupo F", 18223508),
    ("Grupo F", 18346776),
    ("Grupo F", 44810918),
    ("Grupo G", 24468241),
    ("Grupo G", 28741323),
    ("Grupo G", 47544767),
    ("Grupo G", 49355335),
    ("Grupo H", 479510),
    ("Grupo H", 528730),
    ("Grupo H", 1273719),
    ("Grupo H", 13707047),
]

# Criar DataFrame base
df_torneio = pd.DataFrame(dados_torneio, columns=["Grupo", "ID do Time"])

# Adicionar Nome do Time usando o dicionário
df_torneio["Nome do Time"] = df_torneio["ID do Time"].map(nomes_por_id)

# Adicionar ID no Grupo
df_torneio["ID no Grupo"] = df_torneio.groupby("Grupo").cumcount() + 1
df_torneio["ID no Grupo"] = df_torneio["ID no Grupo"].astype(str) + "_" + df_torneio["Grupo"].str[-1]

# Reorganizar colunas
df_torneio = df_torneio[["Grupo", "ID do Time", "Nome do Time", "ID no Grupo"]]

df_liberta_grupo_A = df_torneio[df_torneio["Grupo"] == "Grupo A"]
df_liberta_grupo_B = df_torneio[df_torneio["Grupo"] == "Grupo B"]
df_liberta_grupo_C = df_torneio[df_torneio["Grupo"] == "Grupo C"]
df_liberta_grupo_D = df_torneio[df_torneio["Grupo"] == "Grupo D"]
df_liberta_grupo_E = df_torneio[df_torneio["Grupo"] == "Grupo E"]
df_liberta_grupo_F = df_torneio[df_torneio["Grupo"] == "Grupo F"]
df_liberta_grupo_G = df_torneio[df_torneio["Grupo"] == "Grupo G"]
df_liberta_grupo_H = df_torneio[df_torneio["Grupo"] == "Grupo H"]


# Lista de grupos
grupos = {
    "Grupo A": df_liberta_grupo_A,
    "Grupo B": df_liberta_grupo_B,
    "Grupo C": df_liberta_grupo_C,
    "Grupo D": df_liberta_grupo_D,
    "Grupo E": df_liberta_grupo_E,
    "Grupo F": df_liberta_grupo_F,
    "Grupo G": df_liberta_grupo_G,
    "Grupo H": df_liberta_grupo_H,
}

# # Caminho do arquivo
# caminho_excel = "grupos_fase1_libertadores.xlsx"

# # Salvar em Excel com abas separadas
# with pd.ExcelWriter(caminho_excel) as writer:
#     for nome_grupo, df_grupo in grupos.items():
#         df_grupo.to_excel(writer, sheet_name=nome_grupo, index=False)


# Exibir resultado
display(df_liberta_grupo_A)
display(df_liberta_grupo_B)
display(df_liberta_grupo_C)
display(df_liberta_grupo_D)
display(df_liberta_grupo_E)
display(df_liberta_grupo_F)
display(df_liberta_grupo_G)
display(df_liberta_grupo_H)

,Grupo,ID do Time,Nome do Time,ID no Grupo
0,Grupo A,212042,Tatols Beants F.C,1_A
1,Grupo A,1747619,JV5 Tricolor Gaúcho,2_A
2,Grupo A,13951133,JUV. KP,3_A
3,Grupo A,25811332,SERGRILLO,4_A


,Grupo,ID do Time,Nome do Time,ID no Grupo
4,Grupo B,20696550,Dom Camillo68,1_B
5,Grupo B,30267301,Máquina Laranjja,2_B
6,Grupo B,51010813,LISI GREMISTA,3_B
7,Grupo B,5823700,S.E.R. GRILLO,4_B


,Grupo,ID do Time,Nome do Time,ID no Grupo
8,Grupo C,3851966,cartola scheuer17,1_C
9,Grupo C,7017989,dasdoresfc,2_C
10,Grupo C,13913874,Bandoleros FCS,3_C
11,Grupo C,29228373,seralex,4_C


,Grupo,ID do Time,Nome do Time,ID no Grupo
12,Grupo D,117598,A Lenda Super Vasco F.c,1_D
13,Grupo D,186283,FBC Colorado,2_D
14,Grupo D,24856400,Grêmio imortal 36,3_D
15,Grupo D,19033717,Mau Humor F.C.,4_D


,Grupo,ID do Time,Nome do Time,ID no Grupo
16,Grupo E,36359,KillerColorado,1_E
17,Grupo E,14124559,Paulo Virgili FC,2_E
18,Grupo E,18344271,FÚRIA LEON,3_E
19,Grupo E,18642587,Fedato Futebol Clube,4_E


,Grupo,ID do Time,Nome do Time,ID no Grupo
20,Grupo F,387186,DM Studio,1_F
21,Grupo F,18223508,Rolo Compressor ZN,2_F
22,Grupo F,18346776,AZURRA82,3_F
23,Grupo F,44810918,lsauer fc,4_F


,Grupo,ID do Time,Nome do Time,ID no Grupo
24,Grupo G,24468241,Grêmio imortal 37,1_G
25,Grupo G,28741323,Tabajara de Inhaua PB1,2_G
26,Grupo G,47544767,TORRESMO COM PINGA PRO26.1,3_G
27,Grupo G,49355335,A Lenda Super Vascão f.c,4_G


,Grupo,ID do Time,Nome do Time,ID no Grupo
28,Grupo H,479510,TEAM LOPES 99,1_H
29,Grupo H,528730,Gremiomaniasm,2_H
30,Grupo H,1273719,Texas Club 2026,3_H
31,Grupo H,13707047,Super Vasco f.c,4_H


### Definição dos Confrontos das 6 Rodadas da Fase de Grupos da Libertadores



In [9]:
# Lista dos confrontos da 1ª rodada
confrontos_1a_rodada = [
    # Grupo A
    ("Grupo A", "4_A", "2_A"),
    ("Grupo A", "3_A", "1_A"),

    # Grupo B
    ("Grupo B", "4_B", "2_B"),
    ("Grupo B", "3_B", "1_B"),

    # Grupo C
    ("Grupo C", "4_C", "2_C"),
    ("Grupo C", "3_C", "1_C"),

    # Grupo D
    ("Grupo D", "4_D", "2_D"),
    ("Grupo D", "3_D", "1_D"),

    # Grupo E
    ("Grupo E", "4_E", "2_E"),
    ("Grupo E", "3_E", "1_E"),

    # Grupo F
    ("Grupo F", "3_F", "4_F"),
    ("Grupo F", "1_F", "2_F"),

    # Grupo G
    ("Grupo G", "4_G", "2_G"),
    ("Grupo G", "3_G", "1_G"),

    # Grupo H
    ("Grupo H", "4_H", "2_H"),
    ("Grupo H", "3_H", "1_H"),
]

# Transformar em DataFrame
df_confrontos = pd.DataFrame(confrontos_1a_rodada, columns=["Grupo", "Mandante_ID", "Visitante_ID"])

# Junta com df_torneio para buscar dados dos mandantes
df_mandantes = df_torneio.rename(columns={
    "ID no Grupo": "Mandante_ID",
    "Nome do Time": "Mandante_Nome",
    "ID do Time": "Mandante_ID_Time",
})[["Grupo", "Mandante_ID", "Mandante_Nome", "Mandante_ID_Time"]]

# Junta com df_torneio para buscar dados dos visitantes
df_visitantes = df_torneio.rename(columns={
    "ID no Grupo": "Visitante_ID",
    "Nome do Time": "Visitante_Nome",
    "ID do Time": "Visitante_ID_Time",
})[["Grupo", "Visitante_ID", "Visitante_Nome", "Visitante_ID_Time"]]


In [10]:
# Lista dos confrontos da 2ª rodada
confrontos_2a_rodada = [
     # GRUPO A
    ("Grupo A", "1_A", "4_A"),
    ("Grupo A", "2_A", "3_A"),

    # GRUPO B
    ("Grupo B", "1_B", "4_B"),
    ("Grupo B", "2_B", "3_B"),

    # GRUPO C    
    ("Grupo C", "1_C", "4_C"),
    ("Grupo C", "2_C", "3_C"),

    # GRUPO D
    ("Grupo D", "1_D", "4_D"),
    ("Grupo D", "2_D", "3_D"),

    # GRUPO E
    ("Grupo E", "1_E", "4_E"),
    ("Grupo E", "2_E", "3_E"),

    # GRUPO F
    ("Grupo F", "4_F", "1_F"),
    ("Grupo F", "2_F", "3_F"),

    # GRUPO G    
    ("Grupo G", "1_G", "4_G"),
    ("Grupo G", "2_G", "3_G"),

    # GRUPO H
    ("Grupo H", "1_H", "4_H"),
    ("Grupo H", "2_H", "3_H"),
]

confrontos_3a_rodada = [
    # GRUPO A
    ("Grupo A", "4_A", "3_A"),
    ("Grupo A", "2_A", "1_A"),

    # GRUPO B
    ("Grupo B", "4_B", "3_B"),
    ("Grupo B", "2_B", "1_B"),

    # GRUPO C
    ("Grupo C", "4_C", "3_C"),
    ("Grupo C", "2_C", "1_C"),

    # GRUPO D
    ("Grupo D", "4_D", "3_D"),
    ("Grupo D", "2_D", "1_D"),

    # GRUPO E
    ("Grupo E", "4_E", "3_E"),
    ("Grupo E", "2_E", "1_E"),

    # GRUPO F
    ("Grupo F", "4_F", "2_F"),
    ("Grupo F", "3_F", "1_F"),

    # GRUPO G
    ("Grupo G", "4_G", "3_G"),
    ("Grupo G", "2_G", "1_G"),

    # GRUPO H
    ("Grupo H", "4_H", "3_H"),
    ("Grupo H", "2_H", "1_H"),
]

confrontos_4a_rodada = [
    # GRUPO A
    ("Grupo A", "4_A", "1_A"),
    ("Grupo A", "3_A", "2_A"),

    # GRUPO B
    ("Grupo B", "4_B", "1_B"),
    ("Grupo B", "3_B", "2_B"),

    # GRUPO C
    ("Grupo C", "4_C", "1_C"),
    ("Grupo C", "3_C", "2_C"),

    # GRUPO D
    ("Grupo D", "4_D", "1_D"),
    ("Grupo D", "3_D", "2_D"),

    # GRUPO E
    ("Grupo E", "4_E", "1_E"),
    ("Grupo E", "3_E", "2_E"),

    # GRUPO F
    ("Grupo F", "1_F", "4_F"),
    ("Grupo F", "3_F", "2_F"),

    # GRUPO G
    ("Grupo G", "4_G", "1_G"),
    ("Grupo G", "3_G", "2_G"),

    # GRUPO H
    ("Grupo H", "4_H", "1_H"),
    ("Grupo H", "3_H", "2_H"),
]


confrontos_5a_rodada = [
    # GRUPO A
    ("Grupo A", "3_A", "4_A"),
    ("Grupo A", "1_A", "2_A"),

    # GRUPO B
    ("Grupo B", "3_B", "4_B"),
    ("Grupo B", "1_B", "2_B"),

    # GRUPO C
    ("Grupo C", "3_C", "4_C"),
    ("Grupo C", "1_C", "2_C"),

    # GRUPO D
    ("Grupo D", "3_D", "4_D"),
    ("Grupo D", "1_D", "2_D"),

    # GRUPO E
    ("Grupo E", "3_E", "4_E"),
    ("Grupo E", "1_E", "2_E"),

    # GRUPO F
    ("Grupo F", "2_F", "4_F"),
    ("Grupo F", "1_F", "3_F"),

    # GRUPO G
    ("Grupo G", "3_G", "4_G"),
    ("Grupo G", "1_G", "2_G"),

    # GRUPO H
    ("Grupo H", "3_H", "4_H"),
    ("Grupo H", "1_H", "2_H"),
]


confrontos_6a_rodada = [
    # GRUPO A
    ("Grupo A", "1_A", "3_A"),
    ("Grupo A", "2_A", "4_A"),

    # GRUPO B
    ("Grupo B", "1_B", "3_B"),
    ("Grupo B", "2_B", "4_B"),

    # GRUPO C
    ("Grupo C", "1_C", "3_C"),
    ("Grupo C", "2_C", "4_C"),

    # GRUPO D
    ("Grupo D", "1_D", "3_D"),
    ("Grupo D", "2_D", "4_D"),

    # GRUPO E
    ("Grupo E", "1_E", "3_E"),
    ("Grupo E", "2_E", "4_E"),

    # GRUPO F
    ("Grupo F", "2_F", "1_F"),
    ("Grupo F", "4_F", "3_F"),

    # GRUPO G
    ("Grupo G", "1_G", "3_G"),
    ("Grupo G", "2_G", "4_G"),

    # GRUPO H
    ("Grupo H", "1_H", "3_H"),
    ("Grupo H", "2_H", "4_H"),
]


### Geração das Tabelas de Jogos das 6 Rodadas da Fase de Grupos

In [11]:
# Transformar em DataFrame
df_confrontos = pd.DataFrame(confrontos_1a_rodada, columns=["Grupo", "Mandante_ID", "Visitante_ID"])
df_confrontos["Rodada"] = 1  # <<< ADICIONAR ESTA LINHA
df_rodada_1 = df_confrontos.merge(df_mandantes, on=["Grupo", "Mandante_ID"])
df_rodada_1 = df_rodada_1.merge(df_visitantes, on=["Grupo", "Visitante_ID"])


# Transformar em DataFrame
df_confrontos_2 = pd.DataFrame(confrontos_2a_rodada, columns=["Grupo", "Mandante_ID", "Visitante_ID"])
df_confrontos_2["Rodada"] = 2
df_rodada_2 = df_confrontos_2.merge(df_mandantes, on=["Grupo", "Mandante_ID"])
df_rodada_2 = df_rodada_2.merge(df_visitantes, on=["Grupo", "Visitante_ID"])

# Transformar em DataFrame
df_confrontos_3 = pd.DataFrame(confrontos_3a_rodada, columns=["Grupo", "Mandante_ID", "Visitante_ID"])
df_confrontos_3["Rodada"] = 3
df_rodada_3 = df_confrontos_3.merge(df_mandantes, on=["Grupo", "Mandante_ID"])
df_rodada_3 = df_rodada_3.merge(df_visitantes, on=["Grupo", "Visitante_ID"])

# Criar DataFrame da 4ª rodada
df_confrontos_4 = pd.DataFrame(confrontos_4a_rodada, columns=["Grupo", "Mandante_ID", "Visitante_ID"])
df_confrontos_4["Rodada"] = 4
df_rodada_4 = df_confrontos_4.merge(df_mandantes, on=["Grupo", "Mandante_ID"])
df_rodada_4 = df_rodada_4.merge(df_visitantes, on=["Grupo", "Visitante_ID"])

# Quinta rodada
df_confrontos_5 = pd.DataFrame(confrontos_5a_rodada, columns=["Grupo", "Mandante_ID", "Visitante_ID"])
df_confrontos_5["Rodada"] = 5
df_rodada_5 = df_confrontos_5.merge(df_mandantes, on=["Grupo", "Mandante_ID"])
df_rodada_5 = df_rodada_5.merge(df_visitantes, on=["Grupo", "Visitante_ID"])

# Sexta rodada
df_confrontos_6 = pd.DataFrame(confrontos_6a_rodada, columns=["Grupo", "Mandante_ID", "Visitante_ID"])
df_confrontos_6["Rodada"] = 6
df_rodada_6 = df_confrontos_6.merge(df_mandantes, on=["Grupo", "Mandante_ID"])
df_rodada_6 = df_rodada_6.merge(df_visitantes, on=["Grupo", "Visitante_ID"])


### Rodadas da Fase 1 da Libertadores

### Consolidação dos Confrontos da Fase 1 e Exportação para Excel

In [12]:
df_rodadas = pd.concat([
    df_rodada_1,
    df_rodada_2,
    df_rodada_3,
    df_rodada_4,
    df_rodada_5,
    df_rodada_6
], ignore_index=True)

df_rodadas.to_excel("confrontos_fase_1_libertadores.xlsx", index=False)

# Exibir os confrontos da fase 1
display(df_rodadas.head()) 

,Grupo,Mandante_ID,Visitante_ID,Rodada,Mandante_Nome,Mandante_ID_Time,Visitante_Nome,Visitante_ID_Time
0,Grupo A,4_A,2_A,1,SERGRILLO,25811332,JV5 Tricolor Gaúcho,1747619
1,Grupo A,3_A,1_A,1,JUV. KP,13951133,Tatols Beants F.C,212042
2,Grupo B,4_B,2_B,1,S.E.R. GRILLO,5823700,Máquina Laranjja,30267301
3,Grupo B,3_B,1_B,1,LISI GREMISTA,51010813,Dom Camillo68,20696550
4,Grupo C,4_C,2_C,1,seralex,29228373,dasdoresfc,7017989


### Conversão dos Confrontos em JSON e Exportação para Uso no Front-End

In [13]:
# Criar lista de dicionários no formato desejado
confrontos_js = []

for _, row in df_rodadas.iterrows():
    confronto = {
        "grupo": row["Grupo"],
        "rodada": int(row["Rodada"]),
        "mandante": {
            "id": int(row["Mandante_ID_Time"]),
            "nome": row["Mandante_Nome"],
        },
        "visitante": {
            "id": int(row["Visitante_ID_Time"]),
            "nome": row["Visitante_Nome"],
        }
    }
    confrontos_js.append(confronto)

# Converter para JSON formatado
json_str = json.dumps(confrontos_js, indent=2, ensure_ascii=False)

# Salvar como arquivo JS com uma variável global
with open("confrontos_fase1_libertadores.js", "w", encoding="utf-8") as f:
    f.write("const confrontosFase1 = ")
    f.write(json_str)
    f.write(";")


In [14]:
def exibir_confrontos(df_rodadas, rodada=None, grupo=None):
    """
    Filtra e exibe os confrontos por rodada e/ou grupo.
    
    Parâmetros:
    - df_rodadas: DataFrame com todos os confrontos
    - rodada: número da rodada (int ou None para todas)
    - grupo: nome do grupo (str ou None para todos)
    
    Retorna:
    - DataFrame filtrado com as colunas relevantes
    """
    colunas = ["Rodada", "Grupo", "Mandante_Nome", "Visitante_Nome"]
    df_filtrado = df_rodadas.copy()

    df_filtrado["Rodada"] = df_filtrado["Rodada"].astype(str) + "ª Rodada"    

    if rodada is not None:
        df_filtrado = df_filtrado[df_filtrado["Rodada"] == rodada]

    if grupo is not None:
        df_filtrado = df_filtrado[df_filtrado["Grupo"] == grupo]

    return df_filtrado[colunas].sort_values(by=["Grupo", "Rodada"])


In [15]:
# # Exibir todos os confrontos da rodada 1
# display(exibir_confrontos(df_rodadas, rodada=1))

# Exibir todos os confrontos do Grupo C
display(exibir_confrontos(df_rodadas, grupo="Grupo H").head(6))

# # Exibir confrontos do Grupo A na rodada 2
# display(exibir_confrontos(df_rodadas, rodada=1, grupo="Grupo A"))

,Rodada,Grupo,Mandante_Nome,Visitante_Nome
14,1ª Rodada,Grupo H,Super Vasco f.c,Gremiomaniasm
15,1ª Rodada,Grupo H,Texas Club 2026,TEAM LOPES 99
30,2ª Rodada,Grupo H,TEAM LOPES 99,Super Vasco f.c
31,2ª Rodada,Grupo H,Gremiomaniasm,Texas Club 2026
46,3ª Rodada,Grupo H,Super Vasco f.c,Texas Club 2026
47,3ª Rodada,Grupo H,Gremiomaniasm,TEAM LOPES 99


### Cálculo da Classificação por Grupo com Base nas Pontuações do Cartola

In [16]:
TURNO_INICIO = 1  # início do 2º turno: colunas "Rodada 20..38"

def _coluna_rodada_existente(df_pontuacoes_times, rodada_rel, turno_inicio=TURNO_INICIO):
    """
    Resolve a coluna da rodada, priorizando 2º turno:
      1) "Rodada {turno_inicio - 1 + rodada_rel}"  -> ex.: 1 → 20, 2 → 21
      2) "Rodada {rodada_rel}"                     -> fallback (caso só exista a do 1º turno)
    Retorna o nome encontrado ou None.
    """
    # tenta primeiro a coluna “real” do 2º turno
    cand2 = f"Rodada {turno_inicio - 1 + int(rodada_rel)}"
    if cand2 in df_pontuacoes_times.columns:
        return cand2

    # fallback: coluna literal (só use se a do 2º turno não existir)
    cand1 = f"Rodada {int(rodada_rel)}"
    if cand1 in df_pontuacoes_times.columns:
        return cand1

    return None


def classificacao_por_grupo(df_rodadas, df_pontuacoes, turno_inicio=TURNO_INICIO):
    """
    Classificação dos grupos com base nos confrontos e nas pontuações do Cartola.
    Suporta 2º turno (colunas Rodada 20..38) mesmo que o dataframe de jogos use Rodada 1..N.
    Retorna:
      - df_resultado (classificação geral)
      - df_resultado_por_grupo (dict por grupo)
    """
    # Remove linha auxiliar se existir e normaliza chaves
    df_pontuacoes_times = df_pontuacoes.drop(index='Lider_Rodada', errors='ignore').copy()
    if df_pontuacoes_times.index.dtype != "object":
        df_pontuacoes_times.index = df_pontuacoes_times.index.astype(str)
    df_pontuacoes_times.index = df_pontuacoes_times.index.str.strip()
    df_pontuacoes_times.columns = [str(c).strip() for c in df_pontuacoes_times.columns]

    estatisticas = {}
    times_por_grupo = {}

    for _, confronto in df_rodadas.iterrows():
        try:
            rodada_rel = int(confronto["Rodada"])
        except Exception:
            continue

        mandante  = str(confronto["Mandante_Nome"]).strip()
        visitante = str(confronto["Visitante_Nome"]).strip()
        grupo     = str(confronto.get("Grupo", "Série")).strip() or "Série"

        times_por_grupo.setdefault(grupo, set()).update([mandante, visitante])

        # time precisa existir no DF de pontuações
        if mandante not in df_pontuacoes_times.index or visitante not in df_pontuacoes_times.index:
            continue

        coluna_rodada = _coluna_rodada_existente(df_pontuacoes_times, rodada_rel, turno_inicio)
        if coluna_rodada is None:
            # ainda não há pontos (ou coluna) para esta rodada
            continue

        pm_raw = df_pontuacoes_times.at[mandante, coluna_rodada]
        pv_raw = df_pontuacoes_times.at[visitante, coluna_rodada]

        # Ignora só se for NaN; 0 é válido
        if pd.isna(pm_raw) or pd.isna(pv_raw):
            continue

        # Converte para número (evita comparar strings)
        try:
            pm = float(pm_raw)
            pv = float(pv_raw)
        except Exception:
            continue

        # Inicializa estruturas do grupo/time
        if grupo not in estatisticas:
            estatisticas[grupo] = {}
        for t in (mandante, visitante):
            estatisticas[grupo].setdefault(t, {
                "Pontos": 0, "Vitórias": 0, "Empates": 0, "Derrotas": 0,
                "Total_Cartola": 0.0, "Cartola_Sofrido": 0.0
            })

        # Acumula “gols-cartola”
        estatisticas[grupo][mandante]["Total_Cartola"]   += pm
        estatisticas[grupo][mandante]["Cartola_Sofrido"] += pv
        estatisticas[grupo][visitante]["Total_Cartola"]  += pv
        estatisticas[grupo][visitante]["Cartola_Sofrido"]+= pm

        # Resultado em pontos
        if pm > pv:
            estatisticas[grupo][mandante]["Pontos"]   += 3
            estatisticas[grupo][mandante]["Vitórias"] += 1
            estatisticas[grupo][visitante]["Derrotas"]+= 1
        elif pm < pv:
            estatisticas[grupo][visitante]["Pontos"]   += 3
            estatisticas[grupo][visitante]["Vitórias"] += 1
            estatisticas[grupo][mandante]["Derrotas"]  += 1
        else:
            estatisticas[grupo][mandante]["Pontos"]   += 1
            estatisticas[grupo][visitante]["Pontos"]  += 1
            estatisticas[grupo][mandante]["Empates"]  += 1
            estatisticas[grupo][visitante]["Empates"] += 1

    # Montagem do DataFrame (com placeholders zerados se nada foi computado)
    frames = []
    if estatisticas:
        for grupo, times in estatisticas.items():
            df_g = pd.DataFrame({
                "Grupo": grupo,
                "Nome do Time": list(times.keys()),
                "Pontos": [s["Pontos"] for s in times.values()],
                "Vitórias": [s["Vitórias"] for s in times.values()],
                "Empates": [s["Empates"] for s in times.values()],
                "Derrotas": [s["Derrotas"] for s in times.values()],
                "Total Cartola": [s["Total_Cartola"] for s in times.values()],
                "Cartola Sofrido": [s["Cartola_Sofrido"] for s in times.values()],
            })
            df_g["Saldo Cartola"] = df_g["Total Cartola"] - df_g["Cartola Sofrido"]
            frames.append(df_g)
    else:
        for grupo, times in times_por_grupo.items():
            nomes = sorted(t for t in times if t in df_pontuacoes_times.index)
            if not nomes: 
                continue
            df_g = pd.DataFrame({
                "Grupo": grupo,
                "Nome do Time": nomes,
                "Pontos": 0, "Vitórias": 0, "Empates": 0, "Derrotas": 0,
                "Total Cartola": 0.0, "Cartola Sofrido": 0.0
            })
            df_g["Saldo Cartola"] = 0.0
            frames.append(df_g)

    if frames:
        df_resultado = pd.concat(frames, ignore_index=True)
        # Ordenação e posição
        df_resultado = df_resultado.sort_values(
            by=["Grupo", "Pontos", "Vitórias", "Total Cartola", "Saldo Cartola", "Nome do Time"],
            ascending=[True, False, False, False, False, True]
        ).reset_index(drop=True)
        df_resultado["Posição"] = df_resultado.groupby("Grupo").cumcount() + 1
    else:
        df_resultado = pd.DataFrame(columns=[
            "Grupo","Nome do Time","Pontos","Vitórias","Empates","Derrotas",
            "Total Cartola","Cartola Sofrido","Saldo Cartola","Posição"
        ])

    df_resultado_por_grupo = {
        g: df_resultado[df_resultado["Grupo"] == g].reset_index(drop=True)
        for g in df_resultado["Grupo"].unique()
    }

    return df_resultado, df_resultado_por_grupo


### Geração e Exportação da Classificação Final da Fase de Grupos

In [17]:
# Gerar a classificação da fase 1
df_resultado, df_resultado_por_grupo = classificacao_por_grupo(df_rodadas, df_pontuacoes)

# Salvar cada grupo em uma aba do Excel
with pd.ExcelWriter("classificacao_por_grupo_fase_1.xlsx") as writer:
    for grupo, df in df_resultado_por_grupo.items():
        df.to_excel(writer, sheet_name=grupo, index=False)

# Exibir a classificação geral
df_resultado_grupo_A = df_resultado[df_resultado["Grupo"] == "Grupo A"]
df_resultado_grupo_B = df_resultado[df_resultado["Grupo"] == "Grupo B"]
df_resultado_grupo_C = df_resultado[df_resultado["Grupo"] == "Grupo C"]
df_resultado_grupo_D = df_resultado[df_resultado["Grupo"] == "Grupo D"]
df_resultado_grupo_E = df_resultado[df_resultado["Grupo"] == "Grupo E"]
df_resultado_grupo_F = df_resultado[df_resultado["Grupo"] == "Grupo F"]
df_resultado_grupo_G = df_resultado[df_resultado["Grupo"] == "Grupo G"]
df_resultado_grupo_H = df_resultado[df_resultado["Grupo"] == "Grupo H"]

display(df_resultado_grupo_A, df_resultado_grupo_B, df_resultado_grupo_C, df_resultado_grupo_D, df_resultado_grupo_E, df_resultado_grupo_F, df_resultado_grupo_G, df_resultado_grupo_H)


,Grupo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
0,Grupo A,Tatols Beants F.C,3,1,0,0,66.86,54.60,12.26,1
1,Grupo A,SERGRILLO,3,1,0,0,49.00,47.86,1.14,2
2,Grupo A,JUV. KP,0,0,0,1,54.60,66.86,-12.26,3
3,Grupo A,JV5 Tricolor Gaúcho,0,0,0,1,47.86,49.00,-1.14,4


,Grupo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
4,Grupo B,S.E.R. GRILLO,3,1,0,0,84.86,49.76,35.10,1
5,Grupo B,Dom Camillo68,3,1,0,0,72.70,47.86,24.84,2
6,Grupo B,Máquina Laranjja,0,0,0,1,49.76,84.86,-35.10,3
7,Grupo B,LISI GREMISTA,0,0,0,1,47.86,72.70,-24.84,4


,Grupo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
8,Grupo C,Bandoleros FCS,3,1,0,0,74.06,61.56,12.50,1
9,Grupo C,dasdoresfc,3,1,0,0,72.86,60.16,12.70,2
10,Grupo C,cartola scheuer17,0,0,0,1,61.56,74.06,-12.50,3
11,Grupo C,seralex,0,0,0,1,60.16,72.86,-12.70,4


,Grupo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
12,Grupo D,Mau Humor F.C.,3,1,0,0,73.96,53.66,20.30,1
13,Grupo D,A Lenda Super Vasco F.c,3,1,0,0,68.06,64.70,3.36,2
14,Grupo D,Grêmio imortal 36,0,0,0,1,64.70,68.06,-3.36,3
15,Grupo D,FBC Colorado,0,0,0,1,53.66,73.96,-20.30,4


,Grupo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
16,Grupo E,Paulo Virgili FC,3,1,0,0,84.26,63.90,20.36,1
17,Grupo E,KillerColorado,3,1,0,0,72.45,54.16,18.29,2
18,Grupo E,Fedato Futebol Clube,0,0,0,1,63.90,84.26,-20.36,3
19,Grupo E,FÚRIA LEON,0,0,0,1,54.16,72.45,-18.29,4


,Grupo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
20,Grupo F,lsauer fc,3,1,0,0,62.56,38.26,24.30,1
21,Grupo F,DM Studio,3,1,0,0,60.20,59.25,0.95,2
22,Grupo F,Rolo Compressor ZN,0,0,0,1,59.25,60.20,-0.95,3
23,Grupo F,AZURRA82,0,0,0,1,38.26,62.56,-24.30,4


,Grupo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
24,Grupo G,TORRESMO COM PINGA PRO26.1,3,1,0,0,73.76,57.60,16.16,1
25,Grupo G,Tabajara de Inhaua PB1,3,1,0,0,61.96,54.10,7.86,2
26,Grupo G,Grêmio imortal 37,0,0,0,1,57.60,73.76,-16.16,3
27,Grupo G,A Lenda Super Vascão f.c,0,0,0,1,54.10,61.96,-7.86,4


,Grupo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
28,Grupo H,Texas Club 2026,3,1,0,0,68.06,58.96,9.10,1
29,Grupo H,Gremiomaniasm,3,1,0,0,57.45,42.96,14.49,2
30,Grupo H,TEAM LOPES 99,0,0,0,1,58.96,68.06,-9.10,3
31,Grupo H,Super Vasco f.c,0,0,0,1,42.96,57.45,-14.49,4


### Geração do Arquivo classificacao_fase_1.js com as Classificações por Grupo



In [18]:
# Criar estrutura em formato de dicionário para JSON/JS
classificacao_js = {}

for grupo, df in df_resultado_por_grupo.items():
    classificacao_js[grupo] = []
    for _, row in df.iterrows():
        classificacao_js[grupo].append({
            "posicao": int(row["Posição"]),
            "nome": row["Nome do Time"],
            "pontos": int(row["Pontos"]),
            "vitorias": int(row["Vitórias"]),
            "empates": int(row["Empates"]),
            "derrotas": int(row["Derrotas"]),
            "totalCartola": float(row["Total Cartola"]),
            "cartolaSofrido": float(row["Cartola Sofrido"]),
            "saldoCartola": float(row["Saldo Cartola"])
        })

# Converter para JSON formatado
json_str = json.dumps(classificacao_js, indent=2, ensure_ascii=False)

# Salvar como arquivo JS com uma variável global
with open("classificacao_por_grupo_fase_1.js", "w", encoding="utf-8") as f:
    f.write("const classificacaoFase1 = ")
    f.write(json_str)
    f.write(";")


In [19]:
def exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada, grupo=None):
    """
    Exibe os resultados de uma rodada específica, com pontuação e dados dos times.
    """

    if rodada not in df_rodadas["Rodada"].values:
        return pd.DataFrame([{
            "Grupo": grupo or "-",
            "Rodada": rodada,
            "Mandante_Nome": "-",
            "Mandante_Pontos": "-",
            "Visitante_Nome": "-",
            "Visitante_Pontos": "-",
        }])

    df_filtrado = df_rodadas[df_rodadas["Rodada"] == rodada]
    if grupo:
        df_filtrado = df_filtrado[df_filtrado["Grupo"] == grupo]

    resultados = []

    for _, row in df_filtrado.iterrows():
        grupo_ = row["Grupo"]
        mandante = row["Mandante_Nome"]
        visitante = row["Visitante_Nome"]

        pontos_mandante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(mandante, None)
        pontos_visitante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(visitante, None)

        resultados.append({
            "Grupo": grupo_,
            "Rodada": rodada,
            "Mandante_Nome": mandante,
            "Mandante_Pontos": pontos_mandante,
            "Visitante_Nome": visitante,
            "Visitante_Pontos": pontos_visitante
        })

    return pd.DataFrame(resultados)


In [20]:
# Exibir resultados da 2ª rodada
df_resultados_rodada_1 = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=1)

# Exibir apenas os resultados do Grupo B na 1ª rodada
df_resultados_grupo_B = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=1, grupo="Grupo H")

# Exibir
# display(df_resultados_rodada2)
display(df_resultados_rodada_1)

,Grupo,Rodada,Mandante_Nome,Mandante_Pontos,Visitante_Nome,Visitante_Pontos
0,Grupo A,1,SERGRILLO,49.00,JV5 Tricolor Gaúcho,47.86
1,Grupo A,1,JUV. KP,54.60,Tatols Beants F.C,66.86
2,Grupo B,1,S.E.R. GRILLO,84.86,Máquina Laranjja,49.76
3,Grupo B,1,LISI GREMISTA,47.86,Dom Camillo68,72.70
4,Grupo C,1,seralex,60.16,dasdoresfc,72.86
5,Grupo C,1,Bandoleros FCS,74.06,cartola scheuer17,61.56
6,Grupo D,1,Mau Humor F.C.,73.96,FBC Colorado,53.66
7,Grupo D,1,Grêmio imortal 36,64.70,A Lenda Super Vasco F.c,68.06
8,Grupo E,1,Fedato Futebol Clube,63.90,Paulo Virgili FC,84.26
9,Grupo E,1,FÚRIA LEON,54.16,KillerColorado,72.45


In [21]:
# Criar arquivo com uma aba para cada rodada contendo os resultados detalhados
from pathlib import Path

# Caminho do arquivo de saída
caminho_resultados = "resultados_fase_1.xlsx"

# Descobrir as rodadas únicas no DataFrame
rodadas_disponiveis = sorted(df_rodadas["Rodada"].unique())

with pd.ExcelWriter(caminho_resultados) as writer:
    for rodada in rodadas_disponiveis:
        df_resultados = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=rodada)
        nome_aba = f"Rodada {rodada}"
        df_resultados.to_excel(writer, sheet_name=nome_aba, index=False)

print(f"Arquivo salvo com sucesso: {Path(caminho_resultados).resolve()}")


Arquivo salvo com sucesso: C:\Users\ferna\Projetos\GitHub\cartola_2026\libertadores\datasets_liberta\resultados_fase_1.xlsx


In [22]:
display(df_resultados)

,Grupo,Rodada,Mandante_Nome,Mandante_Pontos,Visitante_Nome,Visitante_Pontos
0,Grupo A,6,Tatols Beants F.C,None,JUV. KP,None
1,Grupo A,6,JV5 Tricolor Gaúcho,None,SERGRILLO,None
2,Grupo B,6,Dom Camillo68,None,LISI GREMISTA,None
3,Grupo B,6,Máquina Laranjja,None,S.E.R. GRILLO,None
4,Grupo C,6,cartola scheuer17,None,Bandoleros FCS,None
5,Grupo C,6,dasdoresfc,None,seralex,None
6,Grupo D,6,A Lenda Super Vasco F.c,None,Grêmio imortal 36,None
7,Grupo D,6,FBC Colorado,None,Mau Humor F.C.,None
8,Grupo E,6,KillerColorado,None,FÚRIA LEON,None
9,Grupo E,6,Paulo Virgili FC,None,Fedato Futebol Clube,None


In [23]:
resultados_js = []

for rodada in sorted(df_rodadas["Rodada"].unique()):
    df_resultados = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=rodada)
    
    for _, row in df_resultados.iterrows():
        resultado = {
            "grupo": row["Grupo"],
            "rodada": int(rodada),
            "mandante": {
                "nome": row["Mandante_Nome"],
                "pontos": float(row["Mandante_Pontos"]) if row["Mandante_Pontos"] is not None else None
            },
            "visitante": {
                "nome": row["Visitante_Nome"],
                "pontos": float(row["Visitante_Pontos"]) if row["Visitante_Pontos"] is not None else None
            },
            "vencedor": (
                "mandante" if row["Mandante_Pontos"] is not None and row["Visitante_Pontos"] is not None and row["Mandante_Pontos"] > row["Visitante_Pontos"]
                else "visitante" if row["Mandante_Pontos"] is not None and row["Visitante_Pontos"] is not None and row["Mandante_Pontos"] < row["Visitante_Pontos"]
                else "empate" if row["Mandante_Pontos"] == row["Visitante_Pontos"] and row["Mandante_Pontos"] is not None
                else "indefinido"
            )

        }
        resultados_js.append(resultado)

# Exportar para arquivo .js
import json

with open("resultados_fase_1.js", "w", encoding="utf-8") as f:
    f.write("const resultadosFase1 = ")
    f.write(json.dumps(resultados_js, indent=2, ensure_ascii=False))
    f.write(";")

# Exporta meta de parcial (usada no front)
try:
    rodada_ref = int(rod_ref)
except Exception:
    rodada_ref = 0

parcial_payload = {"rodada": rodada_ref, "times": {}}
try:
    col_parcial = f"Rodada {rodada_ref}"
    if col_parcial in df_pontuacoes.columns:
        times_map = {}
        for nome in df_pontuacoes.index:
            if nome not in ids_times:
                continue
            try:
                val = df_pontuacoes.at[nome, col_parcial]
            except Exception:
                continue
            if str(val) in ("", "nan"):
                continue
            try:
                times_map[str(ids_times[nome])] = float(val)
            except Exception:
                continue
        parcial_payload["times"] = times_map
except Exception:
    pass

try:
    status_http_val = int(status_http)
except Exception:
    status_http_val = None

liberta_meta = {
    "rodada_atual": rodada_ref,
    "parcial_disponivel": (status_http_val == 2)
}

with open("resultados_fase_1.js", "a", encoding="utf-8") as f:
    f.write("const pontuacaoParcialRodadaAtual = ")
    f.write(json.dumps(parcial_payload, indent=2, ensure_ascii=False))
    f.write(";")
    f.write("window.libertaMeta = ")
    f.write(json.dumps(liberta_meta, indent=2, ensure_ascii=False))
    f.write(";")
    